In [ ]:
%load_ext google.cloud.bigquery

# gnomAD Queries
## Query1
Find 10,000 SNV on chr17 that are more common in women than men, min sample size set to 30.

In [ ]:
%%bigquery result1 --use_rest_api
SELECT DISTINCT 
       start_position AS str_pos,
       reference_bases AS ref,
       alternate_bases.alt AS alt,
       alternate_bases.allele_type AS type,
       vep.SYMBOL AS gene,
       vep.feature_type AS f_type,
       alternate_bases.AC AS AC,
       alternate_bases.AC_female AS AC_f,
       alternate_bases.AC_male AS AC_m,
       ROUND(alternate_bases.AC_female / alternate_bases.AC, 3) AS f_ratio
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE alternate_bases.AC > 30 AND vep.SYMBOL IS NOT NULL
ORDER BY f_ratio DESC
LIMIT 10000


In [12]:
result1.head()

,str_pos,ref,alt,type,gene,f_type,AC,AC_f,AC_m,f_ratio
0,79140505,A,C,snv,AATK,Transcript,41,39,2,0.951
1,79140505,A,C,snv,AATK-AS1,Transcript,41,39,2,0.951
2,33998749,T,TC,ins,AP2B1,Transcript,37,35,2,0.946
3,7221462,T,G,snv,GPS2,Transcript,80,75,5,0.938
4,7221462,T,G,snv,NEURL4,Transcript,80,75,5,0.938


We can condensed the result and only list gene symbols and the number of variants found in the query1.

In [18]:
result1.groupby('gene').count()[['str_pos']].sort_values(by=['str_pos'], ascending=False).head()

,str_pos
gene,
DNAH17,73
CTC-297N7.11,67
RP11-799N11.1,62
RNF213,59
DNAH9,54


## Query2
Find top 1,000 SNV on chr17 that show the most significant differences between male samples of African-American ancestry versus Finnish ancestry

In [ ]:
%%bigquery result2 --use_rest_api
SELECT DISTINCT 
       start_position AS str_pos,
       reference_bases AS ref,
       alternate_bases.alt AS alt,
       alternate_bases.allele_type AS type,
       vep.SYMBOL AS gene,
       vep.feature_type AS f_type,
       alternate_bases.AC_male AS AC_m,
       alternate_bases.AC_fin_male AS AC_fin_m,
       alternate_bases.AC_afr_male AS AC_afr_m,
       ROUND(ABS(alternate_bases.AC_fin_male - alternate_bases.AC_afr_male) / alternate_bases.AC_male, 3) AS fin_afr_diff
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE vep.SYMBOL IS NOT NULL AND
      alternate_bases.AC_male > 20 AND alternate_bases.AC_fin_male > 0 AND alternate_bases.AC_afr_male > 0
order by fin_afr_diff DESC
LIMIT 1000

In [22]:
result2.head()

,str_pos,ref,alt,type,gene,f_type,AC_m,AC_fin_m,AC_afr_m,fin_afr_diff
0,76557038,G,A,snv,DNAH17,Transcript,56,53,1,0.929
1,76116856,C,T,snv,TMC6,Transcript,247,228,1,0.919
2,6900268,G,A,snv,ALOX12,Transcript,34,32,1,0.912
3,6900268,G,A,snv,RP11-589P10.5,Transcript,34,32,1,0.912
4,6900268,G,A,snv,RP11-589P10.7,Transcript,34,32,1,0.912


## Query3
Find top 1000 genes with the highest number of SNV on chr17

In [ ]:
%%bigquery result3 --use_rest_api
SELECT gene, count(1) AS num_snv
FROM
(
SELECT DISTINCT 
       start_position AS str_pos,
       alternate_bases.alt AS alt,
       vep.SYMBOL AS gene,
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE vep.SYMBOL IS NOT NULL AND alternate_bases.allele_type = 'snv'
)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1000

In [24]:
result3.head()

,gene,num_snv
0,CTC-297N7.11,9589
1,DNAH17,9208
2,RP11-799N11.1,9190
3,RNF213,6561
4,DNAH2,6361
